# Doelpuntenklassement Limburg – volledige automatische flow

Werkwijze voor de eindgebruiker:
1. Upload het Excel-bestand met uitslagen en doelpuntenmakers.
2. Het notebook zet dit om naar een tekstbestand met het klassement van de laatste speelronde.
3. Vervolgens upload je het **bron-bestand** met de huidige cumulatieve stand.
4. Het notebook maakt een nieuwe cumulatieve stand en biedt die aan als downloadbaar tekstbestand.

De naam van de uploads (Excel en bron-stand) mag vrij gekozen worden.


# Klassement uit Excel (Limburgse clubs) in Google Colab — V7

Dit notebook:

1. Laat je een Excelbestand uploaden met uitslagen en doelpuntenmakers.
2. Berekent per klasse/divisie een topscorersklassement.
3. Neemt **alleen doelpuntenmakers van clubs uit de provincie Limburg** op (op basis van een vaste lijst).
4. Negeert doelpunten die als **eigen doelpunt** zijn aangeduid (met `eigen doelpunt` of `ed.` in de tekst), maar laat de stand wel doorlopen.
5. Bepaalt per doelpunt de scorende club op basis van de **verandering van de stand** ten opzichte van de vorige stand, inclusief meerdere doelpunten van dezelfde speler in één blok (zoals `0-2, 0-3 en 0-4`).
6. Bundelt alle Limburgse doelpuntenmakers uit **DERDE DIVISIE** en **VIERDE DIVISIE** onder één noemer: `Derde en vierde divisie`.
7. Formatteert gelijke aantallen zoals in een topscorerslijst: alleen bij de **eerste speler in een groep** met hetzelfde doelpuntentotaal wordt het aantal doelpunten vermeld; bij de overige spelers in die groep alleen naam + club.
8. Schrijft de output naar een `.txt`-bestand.
9. Laat je dat `.txt`-bestand downloaden.

Voer de cellen van boven naar beneden uit.


In [ ]:
# Installeer benodigde packages (openpyxl voor Excel)
!pip install openpyxl


In [ ]:
# Upload je Excelbestand (bijv. 'amateurvoetbal.xlsx')

from google.colab import files

print("Upload hier je Excelbestand (bijv. 'amateurvoetbal.xlsx')")
uploaded = files.upload()

# Neem de eerste (en meestal enige) bestandsnaam als inputbestand
EXCEL_PATH = list(uploaded.keys())[0]
print(f"Gebruikt bestand: {EXCEL_PATH}")


In [ ]:
import re
from collections import defaultdict
import openpyxl

# Uitvoerbestand
OUTPUT_PATH = "klassement_output.txt"

# ------------------------------------------------------------------
# LIJST MET LIMBURGSE CLUBS (afgeleid uit het overzicht met [L]-markering)
# ------------------------------------------------------------------

LIMBURG_CLUBS = {
    'Abdissenbosch',
    'Achates',
    'Alfa Sport',
    'America',
    'Amstenrade',
    'BEVO',
    'BMR',
    'BSV Limburgia',
    "BVV'27",
    'Baarlo',
    'Bekkerveld',
    'Belfeldia',
    "Berg'28",
    'Bieslo',
    'Blerick',
    'Boekel Sport',
    'Born',
    'Brevendia',
    'Bunde',
    'Caesar',
    'Chevremont',
    "Conventus'03",
    'DBSV',
    'DES Swalmen',
    'DESM',
    'DEV-Arcen',
    "DFO'20",
    "DVC'16",
    'DVO',
    'Daalhof',
    'De Dem',
    'De Leeuw',
    'De Ster',
    'EMS',
    'EVV',
    'Eijsden',
    'Eikenderveld',
    'Eindse Boys',
    'FC Bemelen',
    'FC Geleen Zuid',
    'FC Gulpen',
    'FC Hoensbroek',
    'FC Kerkrade-West',
    'FC Maasgouw',
    'FC ODA',
    'FC RIA',
    'FC Roerdalen',
    'FCV-Venlo',
    'FSG',
    "GSV'28",
    'Geertruidse Boys',
    'Geulsche Boys',
    'Geusselt Sport',
    'Grashoek',
    'Groene Ster',
    'H.B.S.V.',
    'Haelen',
    'Haslou',
    'Heer',
    'Hegelsom',
    'Heijen',
    'Helden',
    'Hellas',
    'Holthees-Smakt',
    'IVO',
    'IVS',
    'KSV Horn',
    'KVC Oranje',
    'Kakertse Boys',
    'Keer',
    'Koningslust',
    'Kronenberg',
    'Langeberg',
    'Leonidas-W',
    'Leunen',
    'Leveroy',
    'Lindenheuvel-Heidebloem Combinatie',
    'Linne',
    "Lottum-GFC'33",
    "MBC'13",
    'MMC Weert',
    'MSH Maasduinen',
    "MVC'19",
    'Melderslo',
    'Merefeldia',
    'Merselo',
    'Meterik',
    'Milsbeek',
    'Minor',
    'Neerbeek',
    'Oostrum',
    "PEC'20",
    'Partij',
    'Passart-VKC',
    "RIOS'31",
    'RKASV',
    'RKAVC',
    'RKDSO',
    'RKHBS',
    'RKHSV',
    'RKIVV',
    'RKMSV',
    'RKMVC',
    'RKSVB',
    'RKSVN',
    'RKSVO',
    'RKSVV',
    'RKTSV',
    'RKUVC',
    'RKVB',
    'RKVVM',
    'RVU',
    'Reuver',
    'Rimburg',
    'Roggel',
    "Rood Groen LVC'01",
    'Roosteren',
    'SCG',
    'SHH',
    'SNA',
    "SNC'14",
    "SSS'18",
    'SV Brunssum',
    'SV Geuldal',
    'SV Heythuysen',
    'SV Hulsberg',
    'SV Laar',
    'SV Meerssen',
    'SV Simpelveld',
    'SV United',
    'SV Venray',
    'SVC 2000',
    'SVEB-Sporting S.T.',
    "SVH'39",
    'SVM',
    'SVME',
    "SVOC'01",
    'Sanderbout',
    'Schaesberg',
    'Scharn',
    'Schimmert',
    'Schinveld',
    'Sittard',
    'Slekker Boys',
    "Sparta'18",
    'Spaubeek',
    'Sportclub Jekerdal',
    'Sportclub Leeuwen',
    'Sportclub Susteren',
    "Sportclub'25",
    'Sporting H.A.C.',
    'Sporting Heerlen',
    'St. Joost',
    'TSC Irene',
    "UOW'02",
    'Urmondia',
    'VV Hebes',
    'VV Kessel',
    'VV Maastricht West',
    'VV Schaesberg',
    'Vaesrade',
    'Venlosche Boys',
    'Veritas',
    'Vijlen',
    "Vitesse'08",
    'Voerendaal',
    'Walram',
    'Weltania',
    'Wijnandia',
    'Willem I',
    'Wittenhorst',
    'Woander Forest',
    'Ysselsteyn',
    "Zwart-Wit'19",
    'Zwentibold'
}

# Optioneel: verzamel verdachte namen / situaties voor controle
SUSPICIOUS_NAMES = set()

# ------------------------------------------------------------------
# HULPFUNCTIE: NORMALISEER DIVISIENAAM
# ------------------------------------------------------------------

def normalize_division_name(division):
    """
    Bundelt alle DERDE DIVISIE- en VIERDE DIVISIE-reeksen onder één noemer.
    Alles wat 'DERDE DIVISIE' of 'VIERDE DIVISIE' bevat (hoofdletterongevoelig)
    wordt teruggegeven als 'Derde en vierde divisie'.
    Andere klassen/divisies blijven ongewijzigd.
    """
    if not isinstance(division, str):
        return division
    up = division.upper()
    if "DERDE DIVISIE" in up or "VIERDE DIVISIE" in up:
        return "Derde en vierde divisie"
    return division

# ------------------------------------------------------------------
# PARSEN VAN DOELPUNTENMAKERS
# ------------------------------------------------------------------

def parse_goals_cell(text, home_team, away_team):
    """
    Parse de tekst uit de kolom 'DOELPUNTENMAKERS' en geef een lijst terug
    met tuples (spelernaam, clubnaam).

    - Negeert eigen doelpunten (met 'eigen doelpunt' of 'ed.'), maar laat de stand wel doorlopen.
    - Bepaalt de scorende club op basis van het verloop van de stand.
    - Herkent meerdere doelpunten van dezelfde speler in één blok,
      zoals '0-2, 0-3 en 0-4' of '2-0, 3-0, 4-0 en 5-0'.
    """
    global SUSPICIOUS_NAMES

    if not text:
        return []

    pattern = re.compile(r"([^,\.]+?)\s+(\d+-\d+)")
    matches = list(pattern.finditer(text))

    results = []
    last_player_name = None
    last_end = 0
    prev_home, prev_away = 0, 0  # start bij 0-0

    for match in matches:
        raw_name = match.group(1).strip()
        score = match.group(2)

        # ---- 1. Context voor eigen doelpunt-detectie ----
        context = text[last_end:match.end(2)].lower()
        segment = text[match.start(1):match.end(2)].lower()
        last_end = match.end(2)

        # ---- 2. Eigen doelpunt / (ed.) -> stand bijwerken, geen speler ----
        if ("eigen doelpunt" in context
            or "eigen doelpunt" in segment
            or "ed." in context
            or "ed." in segment):
            try:
                home_goals, away_goals = map(int, score.split("-"))
                prev_home, prev_away = home_goals, away_goals
            except ValueError:
                SUSPICIOUS_NAMES.add(f"bad-score-eo:{score}")
            continue

        # ---- 3. Wedstrijdheader overslaan (bevat zowel thuis- als uitclub) ----
        if isinstance(home_team, str) and isinstance(away_team, str):
            if home_team and away_team and home_team in raw_name and away_team in raw_name:
                # Dit is een kopregel / uitslag, geen speler
                continue

        # ---- 4. Naam opschonen (haakjes, 'en', etc.) ----
        name = raw_name

        # Haakjestekst vooraan strippen, bv. '(strafschop) en' -> 'en'
        while name.startswith("(") and ")" in name:
            name = name[name.find(")") + 1 :].strip()

        lower_name = name.lower()

        # 'en' of '0-3 en' = extra doelpunt(en) voor dezelfde speler
        if lower_name == "en" or re.fullmatch(r"\d+-\d+\s+en", lower_name):
            if last_player_name is None:
                SUSPICIOUS_NAMES.add(f"en-without-previous:{name}")
                continue
            name = last_player_name
        else:
            # 'en Arman Melkonjan' -> 'Arman Melkonjan'
            if lower_name.startswith("en "):
                name = name[3:].strip()
            last_player_name = name

        # Verdachte namen loggen (maar niet per se overslaan)
        if not any(c.isalpha() for c in name) or len(name) <= 1:
            SUSPICIOUS_NAMES.add(name)

        # ---- 5. Stand uit de score halen ----
        try:
            home_goals, away_goals = map(int, score.split("-"))
        except ValueError:
            SUSPICIOUS_NAMES.add(f"bad-score:{score}")
            continue

        # ---- 6. Bepaal kant + aantal goals op basis van verloop ----
        scored_side = None  # 'home' of 'away'
        steps = 0

        # Normale stap: precies 1 goal erbij voor één van beide
        if home_goals == prev_home + 1 and away_goals == prev_away:
            scored_side = "home"
            steps = 1
        elif away_goals == prev_away + 1 and home_goals == prev_home:
            scored_side = "away"
            steps = 1
        else:
            # Slimmer: meerdere goals voor dezelfde speler in één blok
            # Voorbeelden: '0-2, 0-3 en 0-4' of '2-0, 3-0, 4-0 en 5-0'
            if name == last_player_name:
                # Meerdere goals voor thuisclub
                if home_goals > prev_home and away_goals == prev_away:
                    scored_side = "home"
                    steps = home_goals - prev_home
                # Meerdere goals voor uitclub
                elif away_goals > prev_away and home_goals == prev_home:
                    scored_side = "away"
                    steps = away_goals - prev_away

        # Nog steeds niets gevonden? Dan is dit een vreemde overgang
        if scored_side is None or steps <= 0:
            SUSPICIOUS_NAMES.add(
                f"weird-score-seq:{prev_home}-{prev_away}->{home_goals}-{away_goals}"
            )
            # Stand wél bijwerken zodat de keten verder kan
            prev_home, prev_away = home_goals, away_goals
            continue

        # ---- 7. Goals toevoegen voor deze speler/club ----
        club = home_team if scored_side == "home" else away_team
        for _ in range(steps):
            results.append((name, club))

        # Stand bijwerken naar de nieuwe score
        prev_home, prev_away = home_goals, away_goals

    return results


def build_rankings(ws):
    """
    Loopt alle blokken in het werkblad 'INVOER' langs en bouwt
    per (genormaliseerde) klasse/divisie een dict met { (speler, club): doelpunten }.

    Let op: DERDE DIVISIE- en VIERDE DIVISIE-reeksen worden gebundeld onder
    'Derde en vierde divisie' via normalize_division_name().
    """
    max_row = ws.max_row
    rankings = {}

    # Zoek kopregels: kolom B = naam klasse, kolom F = 'EINDSTAND'
    header_rows = []
    for r in range(1, max_row + 1):
        if (
            isinstance(ws.cell(r, 2).value, str)
            and ws.cell(r, 2).value
            and ws.cell(r, 6).value == "EINDSTAND"
        ):
            header_rows.append(r)

    for header in header_rows:
        raw_division = ws.cell(header, 2).value.strip()
        division = normalize_division_name(raw_division)
        players = rankings.setdefault(division, defaultdict(int))

        row = header + 1
        while row <= max_row:
            home = ws.cell(row, 2).value
            away = ws.cell(row, 4).value

            # Lege regel = einde van het blok
            if not home and not away:
                break

            scorers = ws.cell(row, 12).value
            if scorers:
                for name, club in parse_goals_cell(str(scorers), str(home), str(away)):
                    players[(name, club)] += 1

            row += 1

        rankings[division] = players

    return rankings


def format_ranking(division, players_dict):
    """
    Zet één (genormaliseerde) klasse/divisie om naar tekst in de gewenste vorm,
    waarbij alleen spelers van LIMBURG_CLUBS worden meegenomen.

    Extra: als meerdere spelers hetzelfde aantal doelpunten hebben, wordt alleen
    bij de eerste speler van de groep het aantal doelpunten vermeld. De overige
    spelers in die groep krijgen alleen naam + club, zonder doelpuntentotaal.
    """
    # Filter: alleen spelers van Limburgse clubs
    filtered_items = {
        (name, club): goals
        for (name, club), goals in players_dict.items()
        if club in LIMBURG_CLUBS
    }

    if not filtered_items:
        return f"{division}\n\n(Geen doelpuntenmakers bij Limburgse clubs)\n\n"

    rows = []
    for (name, club), goals in filtered_items.items():
        last_name = name.split()[-1]
        rows.append(
            {
                "name": name,
                "club": club,
                "goals": goals,
                "last_name": last_name,
            }
        )

    # Sortering: meeste doelpunten, dan achternaam, dan naam, dan club
    rows.sort(
        key=lambda r: (
            -r["goals"],
            r["last_name"].lower(),
            r["name"].lower(),
            r["club"].lower(),
        )
    )

    lines = [division, ""]
    position = 1
    i = 0
    n = len(rows)

    # Spelers met evenveel doelpunten onder dezelfde positie
    while i < n:
        goals_this_group = rows[i]["goals"]
        group = []

        while i < n and rows[i]["goals"] == goals_this_group:
            group.append(rows[i])
            i += 1

        first_in_group = True
        for rec in group:
            prefix = f"{position}. " if first_in_group else ""
            if first_in_group:
                # Enkelvoud/meervoud 'doelpunt(en)' alleen bij eerste in groep
                suffix = "doelpunt" if rec["goals"] == 1 else "doelpunten"
                lines.append(
                    f"{prefix}{rec['name']} ({rec['club']}) - {rec['goals']} {suffix}"
                )
            else:
                # Overige spelers in de groep: alleen naam + club
                lines.append(
                    f"{prefix}{rec['name']} ({rec['club']})"
                )
            first_in_group = False

        position += 1

    return "\n".join(lines) + "\n\n"


def maak_klassement_excel(excel_path=EXCEL_PATH, output_path=OUTPUT_PATH):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    ws = wb["INVOER"]

    rankings = build_rankings(ws)

    with open(output_path, "w", encoding="utf-8") as f:
        for division in rankings:
            f.write(format_ranking(division, rankings[division]))

    print(f"Klassement weggeschreven naar: {output_path}")

    if SUSPICIOUS_NAMES:
        print("\nLet op: er zijn verdachte namen/entries gevonden tijdens het parsen:")
        for n in sorted(SUSPICIOUS_NAMES):
            print(" -", n)


# Voer de functie uit
maak_klassement_excel(EXCEL_PATH, OUTPUT_PATH)


In [ ]:
# Het klassementbestand is gegenereerd
print(f"Klassementbestand aangemaakt op: {OUTPUT_PATH}")
print("Ga nu verder met de volgende stap: upload het bron-bestand voor de cumulatieve stand.")


## Deel 2: Maak de cumulatieve stand op basis van de nieuwe ronde

Het tekstbestand uit deel 1 (`klassement_output.txt`) wordt automatisch gebruikt als invoer
voor dit tweede deel. Je hoeft dit bestand dus niet afzonderlijk te downloaden of te uploaden.
Alleen voor de **bron-stand** volgt nog een uploadstap.


# Doelpuntenklassement Limburg – update notebook

Dit notebook leest:
- een **bronstand** met alle doelpuntenmakers tot nu toe (`bron-stand.txt`)
- een **ronde-output** met de doelpunten van de laatste speelronde (`klassement_output.txt`)

en maakt daaruit een **nieuwe bronstand** (`bron-stand_nieuw.txt`) waarin:
- bestaande spelers hun nieuwe doelpunten erbij krijgen
- nieuwe spelers worden toegevoegd
- de ranglijsten per klasse opnieuw worden opgebouwd (hoogste aantal goals bovenaan, gedeelde posities bij gelijke aantallen, met oplopende rangnummers per doelpuntengroep).

Je kunt dit notebook:
- lokaal in Jupyter draaien, **of**
- in Google Colab gebruiken, met duidelijke upload-knoppen per bestand en een download-knop voor de nieuwe output.


In [ ]:
import re
from collections import defaultdict
from pathlib import Path

try:
    from google.colab import files  # type: ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

BRON_STAND_PATH = Path("bron-stand.txt")
RONDE_OUTPUT_PATH = Path("klassement_output.txt")
NIEUWE_BRON_STAND_PATH = Path("bron-stand_nieuw.txt")


In [ ]:
# ==== HULPFUNCTIES ====

line_pattern = re.compile(
    r"^\s*(?:(\d+)\.\s*)?(.+?)(?:\s*-\s*(\d+)\s+doelpunt(?:en)?)?\s*$",
    re.IGNORECASE,
)

def canonical_group(header: str) -> str:
    h = header.strip()
    hl = h.lower()

    if "derde en vierde divisie" in hl:
        return "Derde en vierde divisie"
    if hl.startswith("eerste klasse"):
        return "Eerste klasse"
    if hl.startswith("tweede klasse"):
        return "Tweede klasse"
    if hl.startswith("derde klasse"):
        return "Derde klasse"
    if hl.startswith("vierde klasse"):
        return "Vierde klasse"
    if hl.startswith("vijfde klasse"):
        return "Vijfde klasse"
    return h

def split_name_club_round(raw_name_club: str):
    raw_name_club = raw_name_club.strip()
    i = raw_name_club.rfind("(")
    j = raw_name_club.rfind(")")
    if i == -1 or j == -1 or j < i:
        return raw_name_club.strip(), None
    name = raw_name_club[:i].strip()
    club = raw_name_club[i+1:j].strip()
    return name, club

def split_name_club_bron(raw_name_club: str):
    raw_name_club = raw_name_club.strip()
    i = raw_name_club.rfind("(")
    j = raw_name_club.rfind(")")
    if i == -1 or j == -1 or j < i:
        return raw_name_club.strip(), None, None
    name = raw_name_club[:i].strip()
    inside = raw_name_club[i+1:j].strip()
    parts = [p.strip() for p in inside.split(",")]
    club = parts[0] if parts else None
    extra = ", ".join(parts[1:]) or None
    return name, club, extra


In [ ]:
# ==== UPLOAD & PARSER VOOR KLASSEMENT_OUTPUT (DEZE SPEELRONDE) ====

if not RONDE_OUTPUT_PATH.exists():
    if IN_COLAB:
        print("Upload nu het bestand met de doelpunten van de laatste speelronde.")
        print("Bijvoorbeeld: 'klassement_output.txt'")
        uploaded = files.upload()  # type: ignore
        fname = list(uploaded.keys())[0]
        RONDE_OUTPUT_PATH = Path(fname)
        print("Geüpload:", ", ".join(uploaded.keys()))
    else:
        print("LET OP: het bestand voor de speelronde ontbreekt op dit pad:", RONDE_OUTPUT_PATH)
        print("Pas RONDE_OUTPUT_PATH aan of kopieer het bestand naar deze map.")

def parse_round_file(path: Path):
    goals_this_round = {}
    groups_for_new = {}
    current_group_header = None
    current_group = None
    last_goals_in_block = None

    with path.open(encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            if "(" not in line:
                current_group_header = line
                current_group = canonical_group(line)
                last_goals_in_block = None
                continue
            m = line_pattern.match(line)
            if not m:
                continue
            _, raw_name_club, goals_str = m.groups()
            name, club = split_name_club_round(raw_name_club)
            if goals_str is not None:
                last_goals_in_block = int(goals_str)
            if last_goals_in_block is None:
                continue
            key = (name, club)
            goals_this_round[key] = goals_this_round.get(key, 0) + last_goals_in_block
            if key not in groups_for_new and current_group is not None:
                groups_for_new[key] = current_group
    return goals_this_round, groups_for_new

if RONDE_OUTPUT_PATH.exists():
    goals_this_round, groups_for_new = parse_round_file(RONDE_OUTPUT_PATH)
    print(f"Ingelezen spelers deze ronde: {len(goals_this_round)}")
else:
    print("Er is nog geen geldig bestand gevonden voor de speelronde.")


In [ ]:
# ==== UPLOAD & PARSER VOOR BRON-STAND (TOTAAL TOT NU TOE) ====

if not BRON_STAND_PATH.exists():
    if IN_COLAB:
        print("Upload nu het bestand met de huidige bron-stand.")
        print("Bijvoorbeeld: 'bron-stand.txt'")
        uploaded = files.upload()  # type: ignore
        fname = list(uploaded.keys())[0]
        BRON_STAND_PATH = Path(fname)
        print("Geüpload:", ", ".join(uploaded.keys()))
    else:
        print("LET OP: het bron-stand-bestand ontbreekt op dit pad:", BRON_STAND_PATH)
        print("Pas BRON_STAND_PATH aan of kopieer het bestand naar deze map.")

def parse_totals_file(path: Path):
    totals_before = {}
    meta_before = {}
    current_group_header = None
    current_group = None
    last_goals_in_block = None

    with path.open(encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            if "(" not in line:
                current_group_header = line
                current_group = canonical_group(line)
                last_goals_in_block = None
                continue
            m = line_pattern.match(line)
            if not m:
                continue
            _, raw_name_club, goals_str = m.groups()
            name, club, extra = split_name_club_bron(raw_name_club)
            if goals_str is not None:
                last_goals_in_block = int(goals_str)
            if last_goals_in_block is None:
                continue
            key = (name, club)
            totals_before[key] = last_goals_in_block
            meta_before[key] = {"group": current_group, "extra": extra}
    return totals_before, meta_before

if BRON_STAND_PATH.exists():
    totals_before, meta_before = parse_totals_file(BRON_STAND_PATH)
    print(f"Ingelezen spelers in bron-stand: {len(totals_before)}")
else:
    print("Er is nog geen geldig bron-stand-bestand gevonden.")


In [ ]:
# ==== SAMENVOEGEN: OUDE STAND + NIEUWE RONDE ====

def merge_totals(totals_before, meta_before, goals_this_round, groups_for_new):
    totals_after = {}
    for key, total in totals_before.items():
        meta = meta_before.get(key, {})
        totals_after[key] = {
            "goals": total,
            "group": meta.get("group"),
            "extra": meta.get("extra"),
        }
    for key, extra_goals in goals_this_round.items():
        if key in totals_after:
            totals_after[key]["goals"] += extra_goals
        else:
            group = groups_for_new.get(key)
            totals_after[key] = {
                "goals": extra_goals,
                "group": group,
                "extra": None,
            }
    return totals_after

try:
    totals_after = merge_totals(
        totals_before=totals_before,
        meta_before=meta_before,
        goals_this_round=goals_this_round,
        groups_for_new=groups_for_new,
    )
    print(f"Totaal spelers in nieuwe stand: {len(totals_after)}")
except NameError:
    print("Kon de totalen niet samenvoegen. Controleer of zowel bron-stand als klassement_output zijn ingelezen.")


In [ ]:
# ==== NIEUWE RANGLIJST OPBOUWEN, WEGSCHRIJVEN EN (IN COLAB) DOWNLOADEN ====

def build_class_block(klassenaam: str, players):
    # Sorteer op goals desc, dan club, dan naam
    players_sorted = sorted(
        players,
        key=lambda p: (-p["goals"], p["club"] or "", p["name"]),
    )

    lines = [klassenaam, ""]

    last_goals = None
    rank_counter = 0  # telt doelpunten-groepen: 1, 2, 3, ...

    for p in players_sorted:
        goals = p["goals"]
        name = p["name"]
        club = p["club"]
        extra = p.get("extra")

        if club and extra:
            inside = f"{club}, {extra}"
        elif club:
            inside = club
        else:
            inside = ""

        parens = f" ({inside})" if inside else ""

        if goals != last_goals:
            rank_counter += 1
            last_goals = goals
            doelpunt_woord = "doelpunt" if goals == 1 else "doelpunten"
            line = f"{rank_counter}. {name}{parens} - {goals} {doelpunt_woord}"
        else:
            line = f"{name}{parens}"

        lines.append(line)

    return lines

try:
    per_class = defaultdict(list)
    for (name, club), info in totals_after.items():
        group = info.get("group") or "Overig"
        per_class[group].append(
            {
                "name": name,
                "club": club,
                "goals": info["goals"],
                "extra": info.get("extra"),
            }
        )

    preferred_order = [
        "Derde en vierde divisie",
        "Eerste klasse",
        "Tweede klasse",
        "Derde klasse",
        "Vierde klasse",
        "Vijfde klasse",
    ]

    ordered_classes = []
    seen = set()

    for g in preferred_order:
        if g in per_class:
            ordered_classes.append(g)
            seen.add(g)

    for g in sorted(per_class.keys()):
        if g not in seen:
            ordered_classes.append(g)
            seen.add(g)

    all_lines = []
    for klassenaam in ordered_classes:
        block_lines = build_class_block(klassenaam, per_class[klassenaam])
        all_lines.extend(block_lines)
        all_lines.append("")

    with NIEUWE_BRON_STAND_PATH.open("w", encoding="utf-8") as out:
        out.write("\n".join(all_lines))

    print(f"Nieuwe bron-stand geschreven naar: {NIEUWE_BRON_STAND_PATH}")

    if IN_COLAB:
        try:
            files.download(str(NIEUWE_BRON_STAND_PATH))  # type: ignore
        except Exception as e:
            print("Automatische download in Colab mislukte:", e)

except NameError:
    print("Kon de nieuwe ranglijst niet opbouwen. Controleer of de eerdere cellen zonder fouten zijn uitgevoerd.")
